# Hunting for credential dumping using mimikatz in windows

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("/home/thedoc/Desktop/Test_logs.csv",index_col=False)

In [3]:
df

,Level,Date and Time,Source,Event ID,Task Category,Values
0,Information,4/2/2020 10:15:41 PM,Microsoft-Windows-Sysmon,10,Process accessed (rule: ProcessAccess),Process accessed:\nRuleName: \nUtcTime: 2020-0...
1,Information,4/2/2020 10:15:21 PM,Microsoft-Windows-Sysmon,10,Process accessed (rule: ProcessAccess),Process accessed:\nRuleName: \nUtcTime: 2020-0...
2,Information,4/2/2020 10:15:01 PM,Microsoft-Windows-Sysmon,10,Process accessed (rule: ProcessAccess),Process accessed:\nRuleName: \nUtcTime: 2020-0...
3,Information,4/2/2020 10:14:59 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-04-...
4,Information,4/2/2020 10:14:57 PM,Microsoft-Windows-Sysmon,8,CreateRemoteThread detected (rule: CreateRemot...,CreateRemoteThread detected:\nRuleName: \nUtcT...
...,...,...,...,...,...,...
22167,Information,2/13/2020 12:42:23 PM,Microsoft-Windows-Sysmon,5,Process terminated (rule: ProcessTerminate),Process terminated:\nRuleName: \nUtcTime: 2020...
22168,Information,2/13/2020 12:42:23 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-02-...
22169,Information,2/13/2020 12:42:23 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-02-...
22170,Information,2/13/2020 12:42:23 PM,Microsoft-Windows-Sysmon,4,Sysmon service state changed,Sysmon service state changed:\nUtcTime: 2020-0...


In [4]:
#Lets group by Event ID and see
df.groupby('Event ID').size()

Event ID
1      3337
2       114
3      2242
4         9
5        73
6        42
8        15
10     3752
11     3359
12      462
13     2504
16        2
22     6259
255       2
dtype: int64

# Lets hunt for mimikatz usage

In [5]:
# Here we are looking for event ID=1, so lets use a filter
filter1 = df['Event ID'] == 1
proc_create = df[filter1]

In [6]:
proc_create.iloc[0]

Level                                                  Information
Date and Time                                 4/2/2020 10:14:59 PM
Source                                    Microsoft-Windows-Sysmon
Event ID                                                         1
Task Category                 Process Create (rule: ProcessCreate)
Values           Process Create:\nRuleName: \nUtcTime: 2020-04-...
Name: 3, dtype: object

In [7]:
# Since the dataset I am using is export from Event Viewer in CSV format
# Looks like most of the data is put in the Values Field, lets explore
proc_create.iloc[0]['Values']

'Process Create:\nRuleName: \nUtcTime: 2020-04-03 03:14:59.595\nProcessGuid: {4ce97d7e-aa33-5e86-0000-0010b698ce01}\nProcessId: 780\nImage: C:\\Program Files\\WindowsApps\\Microsoft.MicrosoftOfficeHub_18.1910.1283.0_x64__8wekyb3d8bbwe\\LocalBridge.exe\nFileVersion: 18.1910.1283.0\nDescription: LocalBridge\nProduct: LocalBridge\nCompany: \nOriginalFileName: LocalBridge.exe\nCommandLine: "C:\\Program Files\\WindowsApps\\Microsoft.MicrosoftOfficeHub_18.1910.1283.0_x64__8wekyb3d8bbwe\\LocalBridge.exe" /InvokerPRAID: Microsoft.MicrosoftOfficeHub notifications\nCurrentDirectory: C:\\Windows\\system32\\\nUser: DESKTOP-1MBQ44E\\rajnepali\nLogonGuid: {4ce97d7e-b6b4-5e83-0000-002067240500}\nLogonId: 0x52467\nTerminalSessionId: 1\nIntegrityLevel: Medium\nHashes: MD5=6CA89724D19DDB51BEF68F0ED25FE609,SHA256=7B302C043E7B0770E896C1630513FE2A38506E056E00608A9CB74A20FE10DC9F,IMPHASH=00000000000000000000000000000000\nParentProcessGuid: {4ce97d7e-aa33-5e86-0000-0010ac83ce01}\nParentProcessId: 7796\nParen

In [8]:
#Please note that sysmon EventID 1 does not contain sourceImage and TargetImage fields
#As you can see, this data is not well structured, so lets parse this
def parse_data(line):
    arr = []
    arr = line.split('\n')
    for val in arr:
        if "Image" in val:
            str = re.match("Image:\s(.*)",val)
            return str.group(1)

In [9]:
#Lets make sure that our function is working as expected
first = parse_data(proc_create.iloc[0]['Values'])
first

'C:\\Program Files\\WindowsApps\\Microsoft.MicrosoftOfficeHub_18.1910.1283.0_x64__8wekyb3d8bbwe\\LocalBridge.exe'

In [10]:
# Now, I am going to add a column to the dataframe and name it "Image". 
# The values for this field is populated by passing values in "Values" field and applying the parse_data function

proc_create['Image'] = proc_create['Values'].apply(parse_data)
proc_create.iloc[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Level                                                  Information
Date and Time                                 4/2/2020 10:14:59 PM
Source                                    Microsoft-Windows-Sysmon
Event ID                                                         1
Task Category                 Process Create (rule: ProcessCreate)
Values           Process Create:\nRuleName: \nUtcTime: 2020-04-...
Image            C:\Program Files\WindowsApps\Microsoft.Microso...
Name: 3, dtype: object

In [11]:
# You can ignore the warning above

def parse_guid(line):
    arr = []
    arr = line.split('\n')
    for val in arr:
        if "ProcessGuid" in val:
            str = re.match("ProcessGuid:\s(.*)",val)
            return str.group(1)

In [12]:
#I am going to build a processGuid data for later use
proc_create['ProcessGuid'] = proc_create['Values'].apply(parse_guid)
proc_create.iloc[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Level                                                  Information
Date and Time                                 4/2/2020 10:14:59 PM
Source                                    Microsoft-Windows-Sysmon
Event ID                                                         1
Task Category                 Process Create (rule: ProcessCreate)
Values           Process Create:\nRuleName: \nUtcTime: 2020-04-...
Image            C:\Program Files\WindowsApps\Microsoft.Microso...
ProcessGuid                 {4ce97d7e-aa33-5e86-0000-0010b698ce01}
Name: 3, dtype: object

In [13]:
proc_create.groupby('Image').size()

Image
C:\Program Files (x86)\LiteManagerFree - Server\ROMFUSClient.exe                                                                 9
C:\Program Files (x86)\LiteManagerFree - Server\ROMServer.exe                                                                   10
C:\Program Files (x86)\TeamViewer\TeamViewer_Service.exe                                                                         6
C:\Program Files (x86)\freeFTPd\freeFTPdService.exe                                                                              8
C:\Program Files\Autopsy-4.14.0\autopsy\ESEDatabaseView\ESEDatabaseView.exe                                                      5
                                                                                                                              ... 
C:\Windows\System32\wsqmcons.exe                                                                                                55
C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.

In [14]:
#[Note: here i am looking for literal string mimikatz.exe]
#Lets see if there is any mimikatz here
filter4 = proc_create['Image'].str.contains("mimikatz.exe")
proc_lsass = proc_create[filter4]

In [15]:
proc_lsass

,Level,Date and Time,Source,Event ID,Task Category,Values,Image,ProcessGuid
4589,Information,3/31/2020 4:40:39 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-03-...,C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\...,{4ce97d7e-b8d7-5e83-0000-0010da962600}
4645,Information,3/31/2020 4:35:11 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-03-...,C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\...,{4ce97d7e-b78f-5e83-0000-0010efb42300}
4666,Information,3/31/2020 4:34:18 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-03-...,C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\...,{4ce97d7e-b75a-5e83-0000-00107bfa1600}


In [16]:
# I put this here in case you want to extract any fields that might be of interest. Example: Hashes :)

def extract_field(line, string):
    arr = []
    arr = line.split('\n')
    for val in arr:
        if string in val:
            #lets write a regex to extract the absolute filename only
            str1 = re.match(f"{string}:\s(.*)",val)
            return str1.group(1)

In [17]:
data = proc_lsass.iloc[0]

In [18]:
Hash = extract_field(data['Values'],'Hashes')
Hash

'MD5=9CD25CEE26F115876F1592DCC63CC650,SHA256=ECE23612029589623E0AE27DA942440A9B0A9CD4F9681EC866613E64A247969D,IMPHASH=11433E8AD7B8D0937563D07A7F8C36E2'

In [19]:
# You can repeat the same process or add a field to the dataframe as your need

# Lets take another approach
### I prefer this approach as this has low false positive.

This time we will take a look at suspicious processes accessing lsass.exe


In [20]:
# Here we are looking for sysmon event ID 10

filter5 = df['Event ID'] == 10
proc_access = df[filter5]

In [21]:
# Lets add sourceImage and TargetImage to the dataframe
# This will be very useful for the analysis

def parse_data_target(line):
    arr = []
    arr = line.split('\n')
    for val in arr:
        if "TargetImage" in val:
            #lets write a regex to extract the absolute filename only
            str = re.match("TargetImage:\s(.*)",val)
            return str.group(1)

In [22]:
proc_access['TargetImage'] = proc_access['Values'].apply(parse_data_target)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
def parse_data_source(line):
    arr = []
    arr = line.split('\n')
    for val in arr:
        if "SourceImage" in val:
            str = re.match("SourceImage:\s(.*)",val)
            return str.group(1)

In [24]:
proc_access['SourceImage'] = proc_access['Values'].apply(parse_data_source)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
proc_access.iloc[0]

Level                                                  Information
Date and Time                                 4/2/2020 10:15:41 PM
Source                                    Microsoft-Windows-Sysmon
Event ID                                                        10
Task Category               Process accessed (rule: ProcessAccess)
Values           Process accessed:\nRuleName: \nUtcTime: 2020-0...
TargetImage                          C:\Windows\system32\lsass.exe
SourceImage      C:\ProgramData\Microsoft\Windows Defender\plat...
Name: 0, dtype: object

In [26]:
#lets see all the processes that accessed lsass.exe
filter6 = proc_access['TargetImage'] == "C:\Windows\system32\lsass.exe"
proc_access[filter6]
proc_access.iloc[0]

Level                                                  Information
Date and Time                                 4/2/2020 10:15:41 PM
Source                                    Microsoft-Windows-Sysmon
Event ID                                                        10
Task Category               Process accessed (rule: ProcessAccess)
Values           Process accessed:\nRuleName: \nUtcTime: 2020-0...
TargetImage                          C:\Windows\system32\lsass.exe
SourceImage      C:\ProgramData\Microsoft\Windows Defender\plat...
Name: 0, dtype: object

In [27]:
proc_access.groupby('SourceImage').size()

SourceImage
C:\ProgramData\Microsoft\Windows Defender\platform\4.18.1911.3-0\MsMpEng.exe    2822
C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\mimikatz.exe                         1
C:\Windows\System32\svchost.exe                                                    5
C:\Windows\system32\svchost.exe                                                  884
C:\Windows\system32\wbem\wmiprvse.exe                                             40
dtype: int64

In [28]:
#Okay lets find out more, you could parse more fields as necessary

filter8 = proc_access['SourceImage'] == r"C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\mimikatz.exe"
proc_mimikatz = proc_access[filter8]

In [29]:
proc_mimikatz

,Level,Date and Time,Source,Event ID,Task Category,Values,TargetImage,SourceImage
4588,Information,3/31/2020 4:40:56 PM,Microsoft-Windows-Sysmon,10,Process accessed (rule: ProcessAccess),Process accessed:\nRuleName: \nUtcTime: 2020-0...,C:\Windows\system32\lsass.exe,C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\...


# Okay, Now lets find the hash of this process so we can use it for correlation to running processes and analyze the data

In [30]:
# We are going to use dataframe proc_lsass for this since this contains the guid. 
# But first lets get the guid for this process.

In [31]:
proc_mimikatz.iloc[0]['Values']

'Process accessed:\nRuleName: \nUtcTime: 2020-03-31 21:40:56.714\nSourceProcessGUID: {4ce97d7e-b8d7-5e83-0000-0010da962600}\nSourceProcessId: 4288\nSourceThreadId: 4320\nSourceImage: C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe\nTargetProcessGUID: {4ce97d7e-b6ad-5e83-0000-001006b40000}\nTargetProcessId: 604\nTargetImage: C:\\Windows\\system32\\lsass.exe\nGrantedAccess: 0x1010\nCallTrace: C:\\Windows\\SYSTEM32\\ntdll.dll+9c584|C:\\Windows\\System32\\KERNELBASE.dll+2730e|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+b5b7a|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+b5ee1|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+b5ab1|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+83384|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+831bc|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+82f99|C:\\Users\\rajnepali\\Desktop\\mimikatz_trunk\\x64\\mimikatz.exe+bba79|C:\\Windo

In [32]:
# Lets use extract field :)

mimikatz_guid = extract_field(proc_mimikatz.iloc[0]['Values'],'SourceProcessGUID')
mimikatz_guid

'{4ce97d7e-b8d7-5e83-0000-0010da962600}'

In [33]:
#Now lets use this guid to find the hash from the proc_lsass dataframe from previous method

filter9 = proc_lsass['ProcessGuid'] == "{4ce97d7e-b8d7-5e83-0000-0010da962600}"
proc_suspicious = proc_lsass[filter9]

In [34]:
proc_suspicious

,Level,Date and Time,Source,Event ID,Task Category,Values,Image,ProcessGuid
4589,Information,3/31/2020 4:40:39 PM,Microsoft-Windows-Sysmon,1,Process Create (rule: ProcessCreate),Process Create:\nRuleName: \nUtcTime: 2020-03-...,C:\Users\rajnepali\Desktop\mimikatz_trunk\x64\...,{4ce97d7e-b8d7-5e83-0000-0010da962600}


In [35]:
# Now lets use the extract_field function again :)

proc_suspicious_hash = extract_field(proc_suspicious.iloc[0]['Values'],'Hashes')
proc_suspicious_hash

'MD5=9CD25CEE26F115876F1592DCC63CC650,SHA256=ECE23612029589623E0AE27DA942440A9B0A9CD4F9681EC866613E64A247969D,IMPHASH=11433E8AD7B8D0937563D07A7F8C36E2'

In [36]:
# You can submit this hash to Virustotal or something similar